In [1]:
import requests
import pandas as pd
from pyspark.sql import SparkSession

spark_session = SparkSession.builder \
    .appName("PySpark Stock Ohlcs") \
    .config("spark.jars", "/Users/matthewyip/bootcamp-python/bootcamp-python/postgresql-42.7.7.jar") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://localhost:5432/bootcamp_2504p"

connection_properties = {
    "user": "postgres",
    "password": "admin1234",
    "driver": "org.postgresql.Driver"
}

symbol="MSFT"
start_period="1657237004" # ~2022
end_period="1751931404" # ~2025
url="https://query1.finance.yahoo.com/v8/finance/chart/" + symbol \
  + "?period1=" + start_period \
  + "&period2=" + end_period \
  + "&interval=1d&events=history"
print(url)

headers = {
  "User-Agent": "Mozilla/5.0" # (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3
}
response = requests.get(url, headers=headers)
data = response.json()
# print(data)
# print(type(data))

result =data["chart"]["result"][0]  # MSFT

timestamp = pd.Series(result["timestamp"])
quote = result["indicators"]["quote"][0]


lows = pd.Series(quote["low"])
highs = pd.Series(quote["high"])
opens = pd.Series(quote["open"])
closes = pd.Series(quote["close"])
volumes = pd.Series(quote["volume"])

# Validation
# 1. count
isCountValid = (len(timestamp) == len(lows) == len(highs) == len(opens) == len(closes) == len(volumes))

if not isCountValid:
    print("Count invalid.")
    exit()
# 2. check for null values
# list -> Series -> isnull().any()
if timestamp.isnull().any() \
  or lows.isnull().any() \
  or highs.isnull().any() \
  or opens.isnull().any() \
  or closes.isnull().any() \
  or volumes.isnull().any():
  print("Data Invalid: Missing data.")
  exit()

converted_dates = pd.to_datetime(timestamp, unit="s").dt.date
# print(converted_dates)

# Many series -> A DataFrame
df = pd.DataFrame({
    "symbol": symbol,
    "date": converted_dates,
    "low": lows,
    "high": highs,
    "open": opens,
    "close": closes,
    "volume": volumes
})
# print(df.head())

# Spark session -> Spark DataFrame
spark_df = spark_session.createDataFrame(df)
# print(spark_df.show())


# write into database
spark_df.write.jdbc(
   url = jdbc_url,
   table = "stock_ohlcs",
   mode = "append",
   properties = connection_properties
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/13 13:42:37 WARN Utils: Your hostname, Yuns-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 172.20.10.7 instead (on interface en0)
25/08/13 13:42:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/08/13 13:42:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/13 13:42:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


https://query1.finance.yahoo.com/v8/finance/chart/MSFT?period1=1657237004&period2=1751931404&interval=1d&events=history


25/08/13 13:42:42 ERROR Executor: Exception in task 5.0 in stage 0.0 (TID 5) 12]
java.sql.BatchUpdateException: Batch entry 0 INSERT INTO stock_ohlcs ("symbol","date","low","high","open","close","volume") VALUES (('MSFT'),('2023-09-29 +08'),('314.9800109863281'::double precision),('319.4700012207031'::double precision),('317.75'::double precision),('315.75'::double precision),('24147300'::int8)) was aborted: ERROR: relation "stock_ohlcs" does not exist
  Position: 13  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:52)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2422)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:580)
	at org.postgresql.jdbc.PgStatement.internalExecuteBatch(PgStatement.java:886)
	at org.postgresql.jdbc.PgStatement.executeBatch(

Py4JJavaError: An error occurred while calling o57.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 0.0 failed 1 times, most recent failure: Lost task 6.0 in stage 0.0 (TID 6) (172.20.10.7 executor driver): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO stock_ohlcs ("symbol","date","low","high","open","close","volume") VALUES (('MSFT'),('2023-12-28 +08'),('374.1600036621094'::double precision),('376.4599914550781'::double precision),('375.3699951171875'::double precision),('375.2799987792969'::double precision),('14327000'::int8)) was aborted: ERROR: relation "stock_ohlcs" does not exist
  Position: 13  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:52)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2422)
	at org.postgresql.

25/08/13 17:56:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 678235 ms exceeds timeout 120000 ms
25/08/13 17:56:42 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/13 17:56:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$